<center><img src="img/Marca-ITBA-Color-ALTA.png" width="250">

<h1>Master en Management & Analytics</h1>
</center>


## Clase 4 - Optimización de hiperparámetros

#### Referencias y bibliografía de consulta:

- Introduction to Machine Learning with Python by Andreas C. Müller and Sarah Guido (O’Reilly) 2017 
- An Introduction to Statistical Learning with Applications in R by Gareth James, Daniela Witten, Trevor Hastie and Robert Tibshirani (Springer) 2017 
- Python Machine Learning - Second Edition by Sebastian Raschka (Packt) 2017
- Hands-On Machine Learning with Scikit-Learn and TensorFlow: Concepts, Tools, and Techniques to Build Intelligent Systems - Aurélien Géron - 2017
- https://scikit-learn.org/


<img src="img/House sale.jpg" width="400"  >

En la clase de hoy vamos a seguir trabajando con el [dataset](https://geodacenter.github.io/data-and-lab/KingCounty-HouseSales2015/) de precios de venta de viviendas del King County. 

Recordemos cuáles son los atributos del dataset y qué representa cada uno:
Tenemos 21 atributos:
- **id**: identificación
- **date**: fecha de venta       
- **price**: precio de venta. Esta es nuestra `variable objetivo`.
- **bedrooms**: cantidad de habitaciones
- **bathrooms**: cantidad de baños
- **sqft_living**: tamaño de la zona habitable en pies cuadrados
- **sqft_lot**: tamaño del lote en pies cuadrados
- **floors**: cantidad de pisos
- **waterfront**: 1 si la propiedad tiene un frente de agua, 0 si no.
- **view**: un índice de 0 a 4 de lo buena que era la vista de la propiedad
- **condition**: estado de la casa, clasificado del 1 al 5
- **grade**: clasificación según la calidad de la construcción, que se refiere a los tipos de materiales utilizados y a la calidad de la mano de obra. Los edificios de mejor calidad (mayor grado) cuestan más de construir por unidad de medida y tienen mayor valor.
- **sqft_above**: Pies cuadrados sobre el suelo
- **sqft_basement**: Pies cuadrados bajo tierra
- **yr_built**: año de construcción
- **yr_renovated**: año de renovación. 0 si no se ha renovado nunca
- **zipcode**: código postal de 5 dígitos
- **lat**: latitud
- **long**: longitud
- **sqft_living15**: tamaño medio del espacio habitable de las 15 casas más cercanas, en pies cuadrados
- **sqft_lot15**: tamaño medio del lote de las 15 casas más cercanas, en pies cuadrados

Vamos a comenzar, como de costumbre importando algunas de las librerías que vamos a necesitar:

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import warnings
warnings.filterwarnings('ignore')

Creamos un DataFrame tomando el csv con los datos:

In [ ]:
dataset = pd.read_csv('data/kc_house_data.csv')

dataset.head()

Vamos a realizar las tareas de limpieza que hicimos la clase pasada:

In [ ]:
# Casteamos el atributo 'date' a datetime
dataset['date'] = pd.to_datetime(dataset['date'])

# Creamos una variable que represente el año de venta
dataset['year_sold'] = dataset['date'].dt.year

# Corregimos el error de carga en 'bedrooms'
dataset.loc[dataset['bedrooms']==33,'bedrooms'] = 3

Creamos nuestra matriz de features y nuestro vector target.

In [ ]:
# Creamos X e y
features_cols = [x for x in dataset.columns if x not in ['price','id', 'date','zipcode']]
X = dataset[features_cols]
y = dataset['price']

# Verificamos el shape y el tipo de X e y:
print("Shape X: {}".format(X.shape))
print("Type X: {}".format(type(X)))
print("Shape y: {}".format(y.shape))
print("Type y: {}".format(type(y)))

Hacemos el split entre train y test sets:

In [ ]:
rs=1

X_train, X_test, y_train, y_test = \
         train_test_split(X, y, test_size=0.3, random_state=rs)

Realizamos la imputación de las variables 'bedrooms' y 'bathrooms' para las casas con valores igual a 0, cuidando de calcular los valores en el set de entrenamiento y utilizando dichos valores para el train y test sets:

In [ ]:
X_train['bedrooms'].replace(0, np.nan, inplace=True)
X_train['bathrooms'].replace(0, np.nan, inplace=True)
X_test['bedrooms'].replace(0, np.nan, inplace=True)
X_test['bathrooms'].replace(0, np.nan, inplace=True)

X_train.fillna(X_train.groupby('floors').transform('median'), inplace=True)

dicc_bed = X_train.groupby('floors')['bedrooms'].median().apply(np.ceil).to_dict()
dicc_bath = X_train.groupby('floors')['bathrooms'].median().apply(np.ceil).to_dict()

X_test['bedrooms'] = X_test['bedrooms']\
                            .fillna(X_test['floors'].apply(lambda x: dicc_bed.get(x)))

X_test['bathrooms'] = X_test['bathrooms']\
                            .fillna(X_test['floors'].apply(lambda x: dicc_bath.get(x)))

#### Estandarización

Un aspecto metodológico importante es que para aplicar estas técnicas de regularización, tenemos que normalizar los datos. Esto se debe a que, al penalizar los parámentros por su tamaño, la escala en la que están medidas las variables se vuelve absolutamente relevante.

Con el modelo de regresión lineal sin regularización, el valor de los parámetros compensaba por las unidades de medida de las variables, por lo que no afectaba al resultado del modelo. Al aplicar regularización, no queremos que efectos de escala afecten la imporancia relativa de los parámetros. Por este motivo, estandarizamos a todas las variables, con media $0$ y desvío estándar $1$, de modo tal que de llevar a todas las variables a una escala común. 

Uno de los aspectos que tenemos que saber antes de entrenar un modelo de machine learning es si requiere o no que los datos estén normalizados.

Apliquemos estandarización, transformando las variables para que tengan media 0 $(\mu = 0)$ y desvío estándar 1 $(\sigma = 1)$, aplicando la fórmula:

$$ x' = \frac{x - \mu}{\sigma}$$

In [ ]:
from sklearn.preprocessing import StandardScaler
stdscaler = StandardScaler()

X_train_std = pd.DataFrame(stdscaler.fit_transform(X_train), columns=X_train.columns)
X_test_std = pd.DataFrame(stdscaler.transform(X_test), columns=X_test.columns)

Es importante notar que estamos haciendo el `fit_transform()` de `stdscaler` con los datos de entrenamiento. Para los datos de testeo, estamos usando el método `transform`, es decir que normalizamos utilizando las medias y desvíos estándar calculados en el set de testo. Esto es así para evitar **data leakage**, es decir filtración de información del set de testo en el modelo.  

Corroboremos los valores de la media y el desvío estándar:

#### Modelo base sin regularización

Vamos a crear un modelo base sin regularización. Para poder apreciar el efecto de la regularización, este modelo debería tener un problema de overfitting. Vamos a tomar nuestra regresión lineal y vamos a crear variables polinómicas para un subset de las $features$, con el objetivo de llegar a tener un problema de sobreajuste que podamos mitigar con las técnicas de regularización que estamos estudiando. 

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

X_train_pol = PolynomialFeatures(degree=3, include_bias=False).fit_transform(\
                      X_train_std[['sqft_living', 'grade', 'sqft_above','sqft_living15',\
                                  'bathrooms', 'view', 'sqft_basement', 'bedrooms']])

X_train_no_pol = X_train_std[[x for x in X_train_std.columns if x not in ['sqft_living', \
                                 'grade', 'sqft_above','sqft_living15',\
                                  'bathrooms', 'view', 'sqft_basement', 'bedrooms']]].values

X_train_feat_eng = np.concatenate((X_train_pol,X_train_no_pol), axis=1 )


print('X_train_pol.shape: {}'.format(X_train_pol.shape))
print('X_train_no_pol.shape: {}'.format(X_train_no_pol.shape))
print('X_train_feat_eng.shape: {}'.format(X_train_feat_eng.shape))

In [ ]:
X_test_pol = PolynomialFeatures(degree=3, include_bias=False).fit_transform(\
                      X_test_std[['sqft_living', 'grade', 'sqft_above','sqft_living15',\
                                  'bathrooms', 'view', 'sqft_basement', 'bedrooms']])

X_test_no_pol = X_test_std[[x for x in X_train_std.columns if x not in ['sqft_living', \
                                 'grade', 'sqft_above','sqft_living15',\
                                  'bathrooms', 'view', 'sqft_basement', 'bedrooms']]].values

X_test_feat_eng = np.concatenate((X_test_pol,X_test_no_pol), axis=1 )


print('X_test_pol.shape: {}'.format(X_test_pol.shape))
print('X_test_no_pol.shape: {}'.format(X_test_no_pol.shape))
print('X_test_feat_eng.shape: {}'.format(X_test_feat_eng.shape))

In [ ]:
lr_base = LinearRegression(fit_intercept=True)

lr_base.fit(X_train_feat_eng, y_train)

y_pred_train = lr_base.predict(X_train_feat_eng)
y_pred_test = lr_base.predict(X_test_feat_eng)

print ('R2 train sin regularización: {}'.format(r2_score(y_train, y_pred_train).round(4)))
print ('R2 test sin regularización: {}'.format(r2_score(y_test, y_pred_test).round(4)))

### Optimización de hiperparámetros

Llegamos al momento de preguntarnos cómo podemos hacer para seleccionar la **combinación óptima de hiperparámentros**.

Por ahora, solamente vimos modelos simples para los que hay que definir algunos pocos hiperparámetros, pero para modelos más complejos vamos a tener que optimizar una gran cantidad de hiperparámetros, por lo que es necesario encontrar una forma de realizar la búsqueda de forma sistemática y poder confiar en los resultados obtenidos. 

### 1) Cross-validation

Antes de avanzar, tenemos que introducir una técnica de validación: el **cross-validation** o **validación cruzada**. 

Cuando hacemos **cross-validation**, en lugar de separar el dataset entre set de entrenamiento y de testeo, **armamos múltiples sets de entrenamiento y validación a partir de un mismo conjunto de datos de entrenamiento**, de forma tal de poder realizar varias pruebas sobre distintos datos desconocidos por el modelo y así tener una mejor noción acerca de su capacidad de generalización.

<img src="img/3foldCV.png" align="center" width="400"/>
<div class='epigraph' align="center"><i>3-fold cross validation</i></div><br>

Este esquema ilustra una estrategia de *cross validation* con 3 particiones o *folds*. Tomamos el dataset de entrenamiento y lo dividimos en tres partes de igual tamaño. En cada prueba, entrenamos con 2/3 de los datos y validamos con las observaciones restantes. 

Una ventaja de este enfoque es que usamos todos los datos para entrenar el modelo, sin necesidad de sacrificar datos en el set de entrenamiento. En general, trabajamos con 3, 5 o 10 *folds* de validación cruzada.

Realizando múltiples *folds* le damos más robustez estadística al análisis ya que promediamos varios resultados con distintos sets de entrenamiento y validación. 

#### Metodología de optimización de hiperparámetros

Vamos a combinar todas las técnicas que vimos para describir la metodología para optimizar hiperparámentros:

<img src="img/gs_crossval.png" align="center" width="600"/>
<div class='epigraph' align="center"><i>Metodología de optimización de hiperparámetros</i></div><br>


- Como primer paso, separamos los datos en set de entrenamiento y de testeo. Usaremos el set de testeo para evaluar el poder de generalización de nuestro modelo.
- Usamos los datos del train set para entrenar el modelo. Seleccionaremos la mejor combinación de hiperparámetros a partir de una estrategia de cross-validation, buscando combinaciones de hiperparámentros con Grid Search o Random Search. 
- Por último, la mejor combinación de hiperparámetros la vamos a volver a entrenar con todo nuestro conjunto de datos de train y vamos a evaluar su poder de generalización con los datos de test que nos guardamos en el primer paso.

#### 1.1) Regresión Ridge con cross-validation

Importemos la clase `Ridge` de `sklearn.linear_model`.

In [ ]:
from sklearn.linear_model import Ridge

`Cross-validaiton` se implementa en `scikit-learn` utilizando la función `cross_val_score` del módulo `model_selection`. Los parámetros de la función `cross_val_score` son el modelo que queremos evaluar, los datos de entrenamiento y el vector con las etiquetas del *ground-truth*.

In [ ]:
from sklearn.model_selection import cross_val_score

Supongamos que queremos evaluar 3 valores de *alpha* en una regresión Ridge. Como vimos en la sección teórica de la clase, no podemos evaluar en el set de testeo, porque en ese caso, la estimación que obtendríamos sobre la capacidad de generalización del modelo ya no sería insesgada. 

Veamos cómo haríamos con `cross-validation`.

In [ ]:
alphas = [0.1, 1, 10, 100, 1000]
dic_scores = dict()

for alpha in alphas:
    rlr = Ridge(alpha=alpha)
    scores = cross_val_score(rlr, X_train_feat_eng, y_train, cv=3)
    dic_scores.update({alpha: scores.mean()})

Observamos el diccionario con los scores:

In [ ]:
dic_scores

Entrenamos una Regresión Ridge en todo el dataset de entrenamiento utilizando el valor óptimo de *alpha*, y ahora evaluamos sobre el set de testeo:

In [ ]:
rlr = Ridge(alpha=max(dic_scores, key=dic_scores.get))

rlr.fit(X_train_feat_eng, y_train)

y_pred_train_ridge = rlr.predict(X_train_feat_eng)
y_pred_test_ridge = rlr.predict(X_test_feat_eng)

print (f"R2 train Ridge: {(r2_score(y_train, y_pred_train_ridge).round(4))}")
print (f"R2 test Ridge: {(r2_score(y_test,y_pred_test_ridge).round(4))}")


### 2) Grid Search y Random Search

Veremos dos grandes métodos (no los únicos) de búsqueda de hiperparámetros:

- **Grid Search**: se busca encontrar el mejor hiperparámetro dentro de una “grilla” (grid) de valores que definimos. Este método realiza una búsqueda exhaustiva dentro del espacio de hiperparámetros definido para buscar la combinación que optimice la función objetivo del modelo. 
- **RandomSearch** dado que el GridSearch implica una búsqueda exhaustiva de todas las combinaciones posibles de la grilla de hiperparámetros especificada, su ejecución pude volverse computacionalmente muy intensa. Con la estrategia de Random Search se realiza la búsqueda de la mejor combinación de hiperparámetros pero a partir de seleccionar en forma aleatoria un subset de los hiperparámetros, lo que achica el espacio de búsqueda.


<img src="img/gsvsrs.png" align="center" width="500"/>
<div class='epigraph' align="center"><i>Grid Search vs. Random Search</i></div><br>

#### 2.1) ElasticNet con Grid Search 

Importemos la clase `ElasticNet` de `sklearn.linear_model`:

In [ ]:
from sklearn.linear_model import ElasticNet

En el caso de `ElasticNet`, tenemos que definir 2 hiperparámetros. Podemos ingresar a la documentación de la clase para ver de qué manera está implementada la combinación entre `Ridge` y `Lasso` y como ingresan ambos hiperparámetros al modelo. 

`ElasticNet` minimiza la siguiente función objetivo:

        1 / (2 * n_samples) * ||y - Xw||^2_2
        + alpha * l1_ratio * ||w||_1
        + 0.5 * alpha * (1 - l1_ratio) * ||w||^2_2

#### Implementamos Grid Search

Vamos a implementar las técnicas de optimización de hiperparámentros. `Sklearn` nos provee clases que nos permiten implementarlas facilmente. 

Para aplicar `Grid Search` en combinación con `cross-validation` disponemos de la clase `GridSearchCV`

In [ ]:
from sklearn.model_selection import GridSearchCV

# Vamos a entrenar una regresión ElasticNet. Tenemos que instanciarla y 
# guardar el modelo en un objeto. 
elr = ElasticNet(max_iter=10000)

# Definimos el espacio de búsqueda de hiperparámetros con un diccionario:
param_grid = {'alpha':[0.9, 1.0, 1.1],
             'l1_ratio':np.linspace(0.4, 0.6, 3)}

print(param_grid)

Instanciamos la clase `GridSearchCV` y la almacenamos en el objeto `grid`. 

Al instanciar la clase le tenemos que pasar:

- El modelo que vamos a optimizar: `elr`
- El espacion de búsqueda de hiperparámetros: `param_grid`
- La estrategia de validación cruzada. Si no pasamos nada, por default serán 5 folds
- El scoring. Por default toma el scoring de la clase del modelo

In [ ]:
from multiprocessing import cpu_count

In [ ]:
X_train_feat_eng = stdscaler.fit_transform(X_train_feat_eng)

In [ ]:
n_jobs = cpu_count() // 2

grid = GridSearchCV(elr, param_grid, cv=3, scoring='r2', n_jobs=n_jobs, verbose=3)
grid.fit(X_train_feat_eng, y_train)

In [ ]:
print(grid.best_estimator_)
print(grid.best_score_)
print(grid.best_params_)

In [ ]:
y_pred_train_grid = grid.predict(X_train_feat_eng)
print ('R2 train GridSearchCV:', r2_score(y_train,y_pred_train_grid).round(5))

y_pred_test_grid = grid.predict(X_test_feat_eng)
print ('R2 test GridSearchCV:', r2_score(y_test,y_pred_test_grid).round(5))

#### Analizamos los resultados del Grid Search

Los resultados del Grid Search se pueden encontrar en el atributo `cv_results_`, que es un diccionario que almacena todos los aspectos de la búsqueda.

In [ ]:
results= pd.DataFrame(grid.cv_results_)

display(results)

Podemos analizar la variación del score en el set de validación al variar los valores de los hiperparámetros con una pivot a la que vamos a visualizar con un mapa de calor:

In [ ]:
scores = results[['mean_test_score', 'param_alpha', 'param_l1_ratio']]
scores = scores.pivot('param_alpha', 'param_l1_ratio', 'mean_test_score')
scores

In [ ]:
sns.heatmap(scores, annot=True, fmt=".3", cmap="viridis");

Vemos que a medida que `alpha` se reduce, mejora el score. Lo contrario sucede con `l1_ratio`. Esto podría indicar que nuestra primera aproximación a `alpha`fue muy alta y, por el contrario, la de `l1_ratio` fue muy baja. 

Utilizemos esta información para buscar mejorar la performance del modelo utilizando `Random Search`.

##### Implementamos Random Search

Para aplicar `Random Search` en combinación con `cross-validation` disponemos de la clase `RandomizedSearchCV`

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

param_random = {'alpha':np.linspace(0.6,0.9,10).round(3),
             'l1_ratio':np.linspace(0.6,0.9,10).round(3)}

In [ ]:
randomcv = RandomizedSearchCV(elr, param_random, n_iter=9, cv=3, scoring='r2',\
                              random_state=15, n_jobs=-1, verbose=1)
randomcv.fit(X_train_feat_eng, y_train)

In [ ]:
print(randomcv.best_estimator_)
print(randomcv.best_score_)
print(randomcv.best_params_)

In [ ]:
y_pred_train_random = randomcv.predict(X_train_feat_eng)
print ('R2 train RandomizedSearchCV:', r2_score(y_train,y_pred_train_random).round(5))

y_pred_test_random = randomcv.predict(X_test_feat_eng)
print ('R2 test RandomizedSearchCV:', r2_score(y_test,y_pred_test_random).round(5))